In [22]:
import nltk
#Необходимо прогнать эти строки при первом запуске
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')

import pymorphy2
import codecs
import string
import re

from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
analyzer = pymorphy2.MorphAnalyzer()
        
rus_filter = re.compile("[а-яА-Я]+")
eng_filter = re.compile("[a-zA-Z]+")



# список символов, которые удаются из текста
MARKS = [',', '.', ':', '?', '«', '»', '-', '(', ')', '!', '\'', "—", ';', "”", "...", "\'\'", "/**//**/",
         "“", "„", "–"]

In [26]:
def parse_words_from_html(file):
    html_page = codecs.open(f"/Users/amourety/Desktop/INFO/WebParserCrawler/src/result/{file}", 'r', 'utf-8')
    
    soup = BeautifulSoup(html_page.read(), features='html.parser')
    # kill all script, style, meta, links, span, a, time, button, li, dt, h2, h3, legend elements
    for script in soup(
            ["script", "style", "meta", "link", "span", "a", "time", "button", "li", "dt", "h2", "h3", "legend"]):
        script.extract()  # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    # убираем знаки препинания
    pattern = r"[^\w]"
    text = re.sub(pattern, " ", text)
    # split to words
    return text.split()

def tokenize():
    words = []
    with open("/Users/amourety/Desktop/INFO/WebParserCrawler/src/result/index.txt", "r") as index:
        lines = index.readlines()
        numbers = [line[: line.find(" ")] for line in lines]
        for num in numbers:
            words.extend([[parse_words_from_html(f"{num}.txt"), num]])
        with open("words.txt", "w", encoding="utf-8") as file:
            for item in words:
                filtered_words = filter_words(item[0])
                for word in filtered_words:
                    file.write(word + ' ' + item[1] + '\n')
                
def filter_words(words):
    words = list(filter(lambda word: (word not in string.punctuation) and (word not in MARKS), words))
    filtered_words = []
    for word in words:
        filtered_words.append("".join(filter(lambda char: char not in MARKS, word)))
    stop_words_rus = set(stopwords.words('russian'))
    stop_words_eng = set(stopwords.words('english'))
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_rus]
    filtered_words = [w.lower() for w in filtered_words if w not in stop_words_eng]
    filtered_words = filter(lambda word: word.isalpha(), filtered_words)
    return filtered_words

def lemmatize():
    with open("words.txt", "r", encoding="utf-8") as words_file:
        lines = words_file.readlines()
        
    rus_lines = [line for line in filter(rus_filter.match, lines)]
    eng_lines = [line for line in filter(eng_filter.match, lines)]
    
    rus_words = []
    eng_words = []
    
    for line in rus_lines:
        rus_words.append(line.strip().split())
    for line in eng_lines:
        eng_words.append(line.strip().split())
    
    lem_dict = {}
    index_dict = {}
    #print(rus_words)
    #print(eng_words)
    
    for word in rus_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_rus(word[0])
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word[0].strip()]
                index_dict[normal_form] = [int(word[1])]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(int(word[1]) not in index_dict[normal_form]):
                    index_dict[normal_form].append(int(word[1]))
                if(word[0] not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word[0].strip())
    for word in eng_words:
        # получаем начальную форму слова
        normal_form = get_normal_form_eng(word[0])
        if normal_form:
            # если такое слово еще не встречалось,создаем ключ с нормальной формой и помещаем само слово как значение
            if normal_form not in lem_dict.keys():
                lem_dict[normal_form] = [word[0].strip()]
                index_dict[normal_form] = [int(word[1])]
            # если такое слово встречалось ранее, добавляем его вариацию в список значений
            else:
                if(int(word[1]) not in index_dict[normal_form]):
                    index_dict[normal_form].append(int(word[1]))
                if(word[0] not in lem_dict[normal_form]):
                    lem_dict[normal_form].append(word[0].strip())
    
    for key in index_dict.keys():
        index_dict[key].sort()
    return(lem_dict, index_dict)

def get_normal_form_rus(word):
    return analyzer.parse(word)[0].normal_form
    
def get_normal_form_eng(word):
    return lemmatizer.lemmatize(word)

# записываем полученные результаты в формате:
# "начальная форма слова: токен токен ..."
# знак ":" служит разделителем между ключом и значениями
def save_lemmatized_tokens_to_file(lem_dict):
    file = open("lemmatized_tokens.txt", "w", encoding="utf-8")
    for word, tokens in lem_dict.items():
        file.write(f"{word}:")
        [file.write(f" {tok}") for tok in set(tokens)]
        file.write("\n")
    file.close()
    
def save_inverted_index_to_file(index_dict):
    file = open("inverted_index.txt", "w", encoding="utf-8")
    for word, indexes in index_dict.items():
        file.write(f"{word}")
        [file.write(f" {index}") for index in set(indexes)]
        file.write("\n")
    file.close()
    
def boolean_search(text, index_dict):
    # убираем знаки препинания
    pattern = r"[^\w]"
    text = re.sub(pattern, " ", text)
    words = text.split()
    words = filter_words(words)
    
    rus_words = [word for word in filter(rus_filter.match, words)]
    eng_words = [word for word in filter(eng_filter.match, words)]
    
    normal_words = []
    
    for word in rus_words:
        normal_words.append(get_normal_form_rus(word))
    for word in eng_words:
        normal_words.append(get_normal_form_eng(word))
    
    page_numbers = []
    for word in normal_words:
        if(word in index_dict.keys()):
            page_numbers.extend(index_dict[word])
    page_numbers.sort()
    return page_numbers

In [27]:
tokenize()

In [28]:
lem_dict, index_dict = lemmatize()

In [29]:
save_lemmatized_tokens_to_file(lem_dict)
save_inverted_index_to_file(index_dict)

In [32]:
text = "Шварц"
page_numbers = boolean_search(text, index_dict)
print(page_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
